In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
###########################
# replacing string function
###########################
def replace_string (file_in, file_out, string_old, string_replaced):
    with open(file_in, "r+") as text_file:
        texts = text_file.read()
        for i in np.arange(len(string_old)):
            texts = texts.replace(string_old[i], string_replaced[i])
    with open(file_out, "w") as text_file:
        text_file.write(texts)


In [ ]:
###########################
# get the evaporation for lake victoria 
###########################
file_names = sorted(glob.glob('../mizuRoute/input/VIC_Lake_Victoria_remapped_*.nc'))

# merge file
merged_dataset = xr.open_mfdataset(file_names)

# number of years
time_variable = merged_dataset['time']
years = time_variable.dt.year
years = np.array(years)
n_years = np.unique(years)
print(n_years)

# find the location index of Lake Victoria
idx = np.array(np.where(ds['ID']==7000016))
idx = idx[0]
    
# slice
evaporation = np.array(merger_dataset['evaporation'][:,idx])
average_evap = sum(evaporation)/n_years

# get the ratio for scale
other_evap = np.array([1550, 1350, 1130, 1520, 1450, 1500, 1370, 1590, 1470, 1475, 1590, 1595, 1400])

# creat the scale factors
scales = other_evap/average_evap


In [ ]:
# Case in which data is located
shutil.copy('../mizuRoute/lake_victoria_temp.submit',
            '../mizuRoute/lake_victoria.submit')

for scale in scales:
    
    #
    replace_string('../mizuRoute/setting/lake_victoria_temp.control',
                   '../mizuRoute/setting/lake_victoria'+f"{float_number:.4f}"+'.control',
                   ['case_temp', 'scale_factor_temp'],
                   ['case'+f"{float_number:.4f}", f"{float_number:.4f}"])
    
    #
    with open('../mizuRoute/lake_victoria.submit', "w") as file:
        line_to_write = 'srun route_runoff.cesm_coupling.exe ../mizuRoute/setting/lake_victoria'+f"{float_number:.4f}"+'.control'+' \n'
        file.write(line_to_write)

